# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive import choice, loguniform
import os
from azureml.core import Environment
import json
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Setup workspace and save workspace parameters as config.json
ws = Workspace.from_config()
experiment_name = 'capstone-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
ws = Workspace.from_config()
# NOTE: Must upload the dataset from local files and name it loan-dataset
found = False
key = "loan-dataset"
description_text = "Dataset of credit risk"

if key in ws.datasets.keys(): 
    found = True
    ds = ws.datasets[key]
else:
    assert key=="loan-dataset",'Need to upload dataset to Azure UI'
df = ds.to_pandas_dataframe()
# save dataset to csv in local directory so it can be used easily by train.py
df.to_csv('loan_dataset.csv')

In [4]:
# Setting up compute as Standard_D2_V2

cluster_name = "compute-project"

## Used some code from train-hyperparameter-tune-deploy-with-keras
# choose a name for your cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-03-21T13:55:50.716000+00:00', 'errors': None, 'creationTime': '2023-03-21T13:28:47.502353+00:00', 'modifiedTime': '2023-03-21T13:28:54.004308+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Using bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
# PARAMETERS OF THE MODEL WE SPECIFIED IN train.py LOGISTIC REGRESSION (NOT A NEURAL NETWORK SO HARD TO MODIFY THIS)
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5, 1, 2),
        '--max_iter': choice(100, 200, 500, 1000),
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = None # No estimator necessary for this project


# Create a ScriptRunConfig Object to specify the configuration details of your training job
if "training" not in os.listdir():
    os.mkdir("./training")

# sklearn environment details provided as the same I used for course 1, so copied it to this project to the current directory as "conda_dependencies.yml" in the current directory
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

args = []

src = ScriptRunConfig(source_directory = './',
                      script='train.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [6]:
#TODO: Submit your experiment
# obtained from Azure's train-hyperparameter-tune-deploy-with-keras notebook
hyperdrive_run = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show() # Showing run details with the widget
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e945f462-8165-46e9-91e9-947f2896d068
Web View: https://ml.azure.com/runs/HD_e945f462-8165-46e9-91e9-947f2896d068?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-228455/workspaces/quick-starts-ws-228455&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-21T15:21:51.439801][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-21T15:21:51.9310056Z][SCHEDULER][INFO]Scheduling job, id='HD_e945f462-8165-46e9-91e9-947f2896d068_0' 
[2023-03-21T15:21:51.9743517Z][SCHEDULER][INFO]Scheduling job, id='HD_e945f462-8165-46e9-91e9-947f2896d068_1' 
[2023-03-21T15:21:52.1062594Z][SCHEDULER][INFO]Scheduling job, id='HD_e945f462-8165-46e9-91e9-947f2896d068_2' 
[2023-03-21T15:21:52.145649][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-21T15:21:52.2197460Z][SCHEDULER][INFO]Scheduling job, id='HD_e945f462-8165-46e9-91e9-947f2896d0

{'runId': 'HD_e945f462-8165-46e9-91e9-947f2896d068',
 'target': 'compute-project',
 'status': 'Completed',
 'startTimeUtc': '2023-03-21T15:21:50.783335Z',
 'endTimeUtc': '2023-03-21T15:26:22.915223Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '13ec4d4f-5460-4d05-bd38-0f8f9db317e8',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '16',
  'score': '0.8583934088568486',
  'best_child_run_id': 'HD_e945f462-8165-46e9-91e9-947f2896d068_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e945f462-8165-46e9-91e9-947f2896d068_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
os.makedirs('./outputs', exist_ok=True)



# Saving best model - had several issues with bestrun.register model with it thinking the output directory didn't exist so saving model right in main directory
model_hyperdrive = best_run.register_model(model_name='hyperdrive_model', model_path = './')

['--C', '0.5', '--max_iter', '200']
['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

